In [0]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
cd /content/gdrive/My\ Drive/Data\ Sets

/content/gdrive/My Drive/Data Sets


In [0]:
!pip3 install sk-video

     |████████████████████████████████| 2.3MB 3.4MB/s 


In [0]:
import skvideo.io
import cv2
import math
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from glob import glob
import skimage
import keras
import keras.models
from keras.applications.vgg19 import VGG19
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.layers import Dense, InputLayer, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D
from keras.preprocessing import image
from tqdm import tqdm
from sklearn.model_selection import train_test_split

ModuleNotFoundError: ignored

In [0]:
base_path = "ucfTrainTestlist/"
f = open(base_path + "trainlist01.txt", "r")
temp = f.read()
train = pd.DataFrame()
train['video_name'] = temp.split('\n')
train = train[:-1]

In [0]:
train_video_tag = []
for i in range(train.shape[0]):
    train_video_tag.append(train['video_name'][i].split('/')[0])
train['tag'] = train_video_tag
train_image_name = []
max_nframes = 6
train_class = []
train_image = []
default_size = (112, 112, 3)

In [0]:
for i in tqdm(range(train.shape[0])):
    videoFile = train['video_name'][i]
    path = 'UCF-101/' + train['tag'][i] + '/' + videoFile.split(' ')[0].split('/')[1]
    z = skvideo.io.vreader(path, num_frames = max_nframes)
    cnt = 0
    for i in z:
        cnt += 1
        train_image.append(skimage.transform.resize(np.squeeze(i[0]), default_size))
        train_image_name.append(videoFile.split(' ')[0].split('/')[1] + '_' + str(cnt))
        train_class.append(videoFile.split(' ')[0].split('/')[1].split('_')[1])

100%|██████████| 9537/9537 [2:34:12<00:00,  1.32s/it]


In [0]:
import gc
train_nw = pd.DataFrame()
train_nw['image'] = train_image_name
train_nw['class'] = train_class
train_image = np.array(train_image)
gc.collect()
train_nw.to_csv('train_new.csv')

In [0]:
y = train_nw['class']
X_train, X_test, y_train, y_test = train_test_split(train_image/255, y, random_state=100, stratify = y, test_size=0.15)
y_train = pd.get_dummies(y_train)
y_test = pd.get_dummies(y_test)

In [0]:
init_model = VGG19(include_top=False, weights='imagenet', input_shape = default_size)
X_train_feat = init_model.predict(X_train)
X_test_feat = init_model.predict(X_test)
X_train_feat = X_train_feat.reshape(X_train_feat.shape[0], X_train_feat.shape[1]*X_train_feat.shape[2]*X_train_feat.shape[3])
X_test_feat = X_test_feat.reshape(X_test_feat.shape[0], X_test_feat.shape[1]*X_test_feat.shape[2]*X_test_feat.shape[3])
mx_train = X_train_feat.max()
X_train_feat /= mx_train
X_test_feat /= mx_train
print(X_train_feat.shape, X_test_feat.shape)





80142336/80134624 [==============================] - 1s 0us/step






(48638, 4608) (8584, 4608)


In [0]:
import gc
gc.collect()
model = keras.models.Sequential([
    Dense(756, activation='relu', input_dim=X_train_feat.shape[1]),
    Dropout(0.4),
    Dense(378, activation='relu'),
    Dropout(0.4),
    Dense(256, activation='relu'),
    Dropout(0.4),
    Dense(128, activation='relu'),
    Dropout(0.4),
    Dense(101, activation='softmax'),
])


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
from keras.callbacks import ModelCheckpoint
mcp_save = ModelCheckpoint('weights.hdf5', save_best_only=True, monitor='val_loss', mode='min')
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy', 'top_k_categorical_accuracy'])
model.fit(X_train_feat, y_train, epochs=200, validation_data=(X_test_feat, y_test), callbacks=[mcp_save], batch_size=256)



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 48638 samples, validate on 8584 samples
Epoch 1/200
48638/48638 [==============================] - 9s 192us/step - loss: 4.5878 - acc: 0.0145 - top_k_categorical_accuracy: 0.0692 - val_loss: 4.5102 - val_acc: 0.0197 - val_top_k_categorical_accuracy: 0.0963
Epoch 2/200
48638/48638 [==============================] - 8s 160us/step - loss: 4.5021 - acc: 0.0207 - top_k_categorical_accuracy: 0.0939 - val_loss: 4.4596 - val_acc: 0.0282 - val_top_k_categorical_accuracy: 0.1167
Epoch 3/200
48638/48638 [==============================] - 8s 161us/step - loss: 4.4417 - acc: 0.0282 - top_k_categorical_accuracy: 0.1135 - val_loss: 4.3775 - val_acc: 0.0393 - val_top_k_categorical_accuracy: 0.1436
Epoch 4/200
48638/48638 [==============================] - 8s 161us/step - loss: 4.3746 - acc: 0.0355 - top_k_categorical_accuracy: 0.1401 - val_loss: 4.2904 - val_acc: 0.0497 - val_top_k_categorical_ac

In [0]:
init_model = InceptionResNetV2(include_top=False, weights='imagenet', input_shape = default_size)
del X_train_feat, X_test_feat, train_image
gc.collect()
X_train_feat = init_model.predict(X_train)
X_test_feat = init_model.predict(X_test)
X_train_feat = X_train_feat.reshape(X_train_feat.shape[0], X_train_feat.shape[1]*X_train_feat.shape[2]*X_train_feat.shape[3])
X_test_feat = X_test_feat.reshape(X_test_feat.shape[0], X_test_feat.shape[1]*X_test_feat.shape[2]*X_test_feat.shape[3])
mx_train = X_train_feat.max()
X_train_feat /= mx_train
X_test_feat /= mx_train
print(X_train_feat.shape, X_test_feat.shape)
model = keras.models.Sequential([
    Dense(756, activation='relu', input_dim=X_train_feat.shape[1]),
    Dropout(0.4),
    Dense(378, activation='relu'),
    Dropout(0.4),
    Dense(256, activation='relu'),
    Dropout(0.4),
    Dense(128, activation='relu'),
    Dropout(0.4),
    Dense(101, activation='softmax'),
])
from keras.callbacks import ModelCheckpoint
mcp_save = ModelCheckpoint('weights2_.hdf5', save_best_only=True, monitor='val_loss', mode='min')
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy', 'top_k_categorical_accuracy'])
model.fit(X_train_feat, y_train, epochs=200, validation_data=(X_test_feat, y_test), callbacks=[mcp_save], batch_size=64)